# Embedding

In [1]:
import csv
import re
from abc import ABC, abstractmethod
import numpy as np

np.random.seed(99)

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = None
        self.gradient_fn = lambda: None
        self.parents = set()

    def gradient(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.gradient()

    def shape(self):
        return self.data.shape

    def size(self):
        return np.prod(self.data.shape[1:])

In [3]:
class DataLoader:

    def __init__(self):
        self.reviews = []
        self.sentiments = []
        with open('reviews.csv', 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)
            for _, row in enumerate(reader):
                self.reviews.append(row[0])
                self.sentiments.append(row[1])

        split_reviews = []
        for r in self.reviews:
            split_reviews.append(self.clean_text(r.lower()).split())

        self.vocabulary = set(w for r in split_reviews for w in r)
        self.word2index = {w: idx for idx, w in enumerate(self.vocabulary)}
        self.index2word = {idx: w for idx, w in enumerate(self.vocabulary)}
        self.tokens = [[self.word2index[w] for w in r if w in self.word2index] for r in split_reviews]

        self.train()

    @staticmethod
    def clean_text(text):
        txt = re.sub(r'<[^>]+>', '', text)
        txt = re.sub(r'[^a-zA-Z0-9\s]', '', txt)
        return txt

    def encode(self, text):
        words = self.clean_text(text.lower()).split()
        return [self.word2index[word] for word in words]

    def decode(self, tokens):
        return " ".join([self.index2word[index] for index in tokens])

    def train(self):
        self.features = [list(set(idx)) for idx in self.tokens[:-10]]
        self.labels = [0 if idx == "negative" else 1 for idx in self.sentiments[:-10]]

    def eval(self):
        self.features = [list(set(idx)) for idx in self.tokens[-10:]]
        self.labels = [0 if idx == "negative" else 1 for idx in self.sentiments[-10:]]

    def __len__(self):  # 3
        return len(self.features)

    def __getitem__(self, index):  # 4
        return (Tensor(self.features[index: index + 1]),
                Tensor(self.labels[index: index + 1]))

In [4]:
class Layer(ABC):

    def __init__(self):
        self.training = True

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    def parameters(self):
        return []

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

In [5]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        super().__init__()
        self.in_size = in_size
        self.out_size = out_size

        self.weight = Tensor(np.random.rand(out_size, in_size) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad = p.grad.T @ x.data
            self.bias.grad = np.sum(p.grad, axis=0)
            x.grad = p.grad @ self.weight.data

        p.gradient_fn = gradient_fn
        p.parents = {self.weight, self.bias, x}
        return p

    def parameters(self):
        return [self.weight, self.bias]

In [6]:
class Sequential(Layer):

    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def forward(self, x: Tensor):
        for l in self.layers:
            x = l(x)
        return x

    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

    def train(self):
        for l in self.layers:
            l.train()

    def eval(self):
        for l in self.layers:
            l.eval()

In [7]:
class Embedding(Layer):

    def __init__(self, vocabulary_size, embedding_size, axis=1):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.axis = axis

        self.weight = Tensor(np.random.rand(embedding_size, vocabulary_size) / vocabulary_size)

    def forward(self, x: Tensor):
        p = Tensor(np.sum(self.weight.data.T[x.data], axis=self.axis))

        def gradient_fn():
            if self.weight.grad is None:
                self.weight.grad = np.zeros_like(self.weight.data)
            self.weight.grad.T[x.data] += p.grad

        p.gradient_fn = gradient_fn
        p.parents = {self.weight}
        return p

    def parameters(self):
        return [self.weight]

In [8]:
class Tanh(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.tanh(x.data))

        def gradient_fn():
            x.grad = p.grad * (1 - p.data ** 2)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

In [9]:
class Sigmoid(Layer):

    def __init__(self, clip_range=(-100, 100)):
        super().__init__()
        self.clip_range = clip_range

    def forward(self, x: Tensor):
        z = np.clip(x.data, self.clip_range[0], self.clip_range[1])
        p = Tensor(1 / (1 + np.exp(-z)))

        def gradient_fn():
            x.grad = p.grad * p.data * (1 - p.data)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

In [10]:
class BCELoss:

    def __call__(self, p: Tensor, y: Tensor):
        clipped = np.clip(p.data, 1e-7, 1 - 1e-7)
        bce = Tensor(-np.mean(y.data * np.log(clipped)
                              + (1 - y.data) * np.log(1 - clipped)))

        def gradient_fn():
            p.grad = (clipped - y.data) / (clipped * (1 - clipped) * len(p.data))

        bce.gradient_fn = gradient_fn
        bce.parents = {p}
        return bce

In [11]:
class SGD:

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def backward(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

In [12]:
LEARNING_RATE = 0.1
EPOCHS = 10

In [13]:
dataset = DataLoader()

model = Sequential([Embedding(len(dataset.vocabulary), 64),
                    Tanh(),
                    Linear(64, 16),
                    Tanh(),
                    Linear(16, 1),
                    Sigmoid()])
loss = BCELoss()
sgd = SGD(model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    for i in range(len(dataset)):
        feature, label = dataset[i]

        prediction = model(feature)
        error = loss(prediction, label)

        error.gradient()
        sgd.backward()

print(f'Prediction: {prediction.data}')
print(f'Error: {error.data}')

Prediction: [[0.99915852]]
Error: 0.0008418313188558764


In [14]:
dataset.eval()

result = 0
for i in range(len(dataset)):
    feature, label = dataset[i]

    prediction = model(feature)
    if np.abs(prediction.data - label.data) < 0.5:
        result += 1
print(f'Result: {result} of {len(dataset)}')

Result: 10 of 10
